In [1]:
# !pip install torch_scatter torch_sparse torch_cluster torch_spline_conv
# !pip install torch_geometric

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm.auto import tqdm
from timeit import default_timer as timer


from torch import nn, optim, Tensor

from torch_sparse import SparseTensor, matmul

from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn.conv import MessagePassing

from sklearn.model_selection import KFold
import gc 
import warnings
warnings.filterwarnings("ignore")

In [3]:
output_path = 'output/models/'

In [4]:
movies = pd.read_csv('Nikolay/dataset/ml-latest-small/movies.csv')
ratings = pd.read_csv('Nikolay/dataset/ml-latest-small/ratings.csv')

directed_by_path = 'Nikolay/dataset/ml-latest-small/directed_by.csv'
stars_path = 'Nikolay/dataset/ml-latest-small/stars.csv'
produced_by_path = 'Nikolay/dataset/ml-latest-small/produced_by.csv'
produced_in_path = 'Nikolay/dataset/ml-latest-small/produced_in.csv'

directed_by_df = pd.read_csv(directed_by_path)[["movieId", "director"]]
stars_df = pd.read_csv(stars_path)[["movieId", "castMember"]]
produced_by_df = pd.read_csv(produced_by_path)[["movieId", "company"]]
produced_in_df = pd.read_csv(produced_in_path)[["movieId", "country"]]

In [6]:
len(movies.movieId.unique())

9742

In [7]:
all_metadata_df = pd.concat([
    directed_by_df.rename(columns={'director': 'metaNodeURI'}),
    stars_df.rename(columns={'castMember': 'metaNodeURI'}),
    produced_by_df.rename(columns={'company': 'metaNodeURI'}),
    produced_in_df.rename(columns={'country': 'metaNodeURI'}),
])
print(len(all_metadata_df))
all_metadata_df.head()

172141


,movieId,metaNodeURI
0,1,Q269214
1,2,Q352948
2,3,Q1138881
3,4,Q182763
4,5,Q376130


In [8]:
len(all_metadata_df.metaNodeURI.unique())

48382

In [9]:
metadata_embeddings = movies[['movieId']]

In [10]:
metadata_embeddings.head()

,movieId
0,1
1,2
2,3
3,4
4,5


In [12]:
node2idx = {}
for i, node in enumerate(all_metadata_df.metaNodeURI.unique()):
    node2idx[node] = i

In [13]:
num_meta_nodes = len(all_metadata_df.metaNodeURI.unique())
num_meta_nodes

48382

In [15]:
movie2embedding = {}
for i, movie_id in enumerate(movies.movieId.unique()):
    embedding = np.zeros(num_meta_nodes, dtype=np.float32)
    for node in all_metadata_df[all_metadata_df['movieId'] == movie_id]['metaNodeURI']:
        embedding[node2idx[node]] = 1
    movie2embedding[movie_id] = embedding

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [18]:
def get_edge_index(X, X_meta, userid2idx, movieid2idx, meta_node_uri2idx):
    edge_index = [[], []]
    edge_values = []
    for i, row in X.iterrows():
        edge_index[0].append(userid2idx.get(row['userId'], 0))
        edge_index[1].append(movieid2idx.get(row['movieId'], 0))
        edge_values.append(row['rating'])
    for _, row in X_meta.iterrows():
        edge_index[0].append(meta_node_uri2idx.get(row['metaNodeURI'], 0))
        edge_index[1].append(movieid2idx.get(row['movieId'], 0))
        edge_values.append(0)

    return edge_index, edge_values

In [19]:
def convert_adj_mat_edge_index_to_r_mat_edge_index(input_edge_index, input_edge_values, num_users, num_meta_nodes, num_movies):

    sparse_input_edge_index = SparseTensor(row=input_edge_index[0],
                                           col=input_edge_index[1],
                                           value = input_edge_values,
                                           sparse_sizes=((num_users + num_meta_nodes + num_movies), num_users + num_meta_nodes + num_movies))

    adj_mat = sparse_input_edge_index.to_dense()
    interact_mat = adj_mat[: num_users + num_meta_nodes, num_users + num_meta_nodes :]

    r_mat_edge_index = interact_mat.to_sparse_coo().indices()
    r_mat_edge_values = interact_mat.to_sparse_coo().values()

    return r_mat_edge_index, r_mat_edge_values

In [20]:
def convert_r_mat_edge_index_to_adj_mat_edge_index(input_edge_index, input_edge_values, num_users, num_meta_nodes, num_movies):
    R = torch.zeros((num_users + num_meta_nodes, num_movies))
    for i in range(len(input_edge_index[0])):
        row_idx = input_edge_index[0][i]
        col_idx = input_edge_index[1][i]
        R[row_idx][col_idx] = input_edge_values[i] 

    R_transpose = torch.transpose(R, 0, 1)

    adj_mat = torch.zeros((num_users + num_meta_nodes + num_movies , num_users + num_meta_nodes + num_movies))
    adj_mat[: num_users + num_meta_nodes, num_users + num_meta_nodes :] = R.clone()
    adj_mat[num_users + num_meta_nodes :, : num_users + num_meta_nodes] = R_transpose.clone()

    adj_mat_coo = adj_mat.to_sparse_coo()
    adj_mat_coo_indices = adj_mat_coo.indices()
    adj_mat_coo_values = adj_mat_coo.values()
    return adj_mat_coo_indices, adj_mat_coo_values

In [21]:
import pickle
with open('Aleksis/movie2abstract_embed.pickle', 'rb') as f:
    movie2abstract_embed = pickle.load(f)

In [22]:
def get_abstract_embeddings(movies: torch.Tensor, idx2movieId):
    result = []

    for movie in movies:
        movieId = idx2movieId[movie.item()]
        result.append(movie2abstract_embed[movieId])
    return torch.tensor(result)

In [23]:
def train_step(model,
               edge_index_train,
               r_mat_edge_index_train,
               r_mat_edge_values_train,
               abstract_embeddings_train,
               meta_nodes_embeddings_train,
               loss_fn,
               optimizer,
               device):

    model.train()
    y_pred = model(edge_index_train, r_mat_edge_index_train, abstract_embeddings_train)

    loss = loss_fn(y_pred, r_mat_edge_values_train.view(-1, 1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()

def test_step(model,
              edge_index_test,
              r_mat_edge_index_test,
              r_mat_edge_values_test,
              abstract_embeddings_test,
              meta_nodes_embeddings_test,
              loss_fn,
              device):
    model.eval()

    with torch.inference_mode():

        y_pred = model(edge_index_test, r_mat_edge_index_test, abstract_embeddings_test)

        loss = loss_fn(y_pred, r_mat_edge_values_test.view(-1, 1))

        return loss.item()

def train(model,
        optimizer,
        loss_fn,
        epochs,
        device,
        scheduler,
        edge_index_train,
        r_mat_edge_index_train,
        r_mat_edge_values_train,
        abstract_embeddings_train,
        meta_nodes_embeddings_train,
        edge_index_test,
        r_mat_edge_index_test,
        r_mat_edge_values_test,
        abstract_embeddings_test,
        meta_nodes_embeddings_test
):
    results = {"train_loss": [],
               "test_loss": [],
    }
    model.to(device)

    best_loss = float('inf')

    edge_index_train = edge_index_train.to(device)
    r_mat_edge_index_train = r_mat_edge_index_train.to(device)
    r_mat_edge_values_train = r_mat_edge_values_train.float().to(device)
    abstract_embeddings_train = abstract_embeddings_train.to(device)
    meta_nodes_embeddings_train = meta_nodes_embeddings_train.to(device)
    edge_index_test = edge_index_test.to(device)
    r_mat_edge_index_test = r_mat_edge_index_test.to(device)
    r_mat_edge_values_test = r_mat_edge_values_test.float().to(device)
    abstract_embeddings_test = abstract_embeddings_test.to(device)
    meta_nodes_embeddings_test = meta_nodes_embeddings_test.to(device)

    for epoch in tqdm(range(epochs)):

        start_time = timer()
        train_loss = train_step(model=model,
                                edge_index_train=edge_index_train,
                                r_mat_edge_index_train=r_mat_edge_index_train,
                                r_mat_edge_values_train=r_mat_edge_values_train,
                                abstract_embeddings_train=abstract_embeddings_train,
                                meta_nodes_embeddings_train=meta_nodes_embeddings_train,
                                loss_fn=loss_fn,
                                optimizer=optimizer,
                                device=device)
        end_time = timer()
        test_loss = test_step(model=model,
                              edge_index_test=edge_index_test,
                              r_mat_edge_index_test=r_mat_edge_index_test,
                              r_mat_edge_values_test=r_mat_edge_values_test,
                              abstract_embeddings_test=abstract_embeddings_test,
                              meta_nodes_embeddings_test=meta_nodes_embeddings_test,
                              loss_fn=loss_fn,
                              device=device)

        results["train_loss"].append(train_loss)
        results["test_loss"].append(test_loss)

        if epoch % 200 == 0 and epoch != 0:
            scheduler.step()

            if best_loss > test_loss:
                best_loss = test_loss
                # print('Saving new best model.')
                model.save( output_path + model.name + ".pt")

    return results

def plot_loss_curves(results):

    loss = results["train_loss"]
    test_loss = results["test_loss"]

    epochs = range(len(results["train_loss"]))

    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label="train_loss")
    plt.plot(epochs, test_loss, label="test_loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend()

In [24]:
# defines LightGCN model
class DeepLightGCN(MessagePassing):

    def save(self, fileName):
        torch.save(self.state_dict(), fileName)

    def load(self, fileName, device):
        self.load_state_dict(torch.load(fileName, map_location=device))

    def __init__(self, num_users, num_items, num_meta_nodes, embedding_dim=64, K=3, abstract_embedding_dim=384, n_hidden=32, add_self_loops=False, dropout_rate=0.1, name='name'):

        super().__init__()
        self.dropout_rate = dropout_rate
        self.num_users = num_users
        self.num_items = num_items
        self.num_meta_nodes = num_meta_nodes
        self.embedding_dim = embedding_dim
        self.K = K
        self.add_self_loops = add_self_loops
        self.name = name
        self.drop1 = nn.Dropout(dropout_rate)
        self.drop2 = nn.Dropout(dropout_rate)
        self.abstract_embedding_dim = abstract_embedding_dim

        self.users_emb = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.embedding_dim) 

        self.items_emb = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.embedding_dim) 

        self.meta_nodes_emb = nn.Embedding(num_embeddings=self.num_meta_nodes, embedding_dim=self.embedding_dim)

        nn.init.normal_(self.users_emb.weight, std=0.1)
        nn.init.normal_(self.items_emb.weight, std=0.1)
        nn.init.normal_(self.meta_nodes_emb.weight, std=0.1)

        self.lin1 = nn.Linear(embedding_dim*2 + abstract_embedding_dim, n_hidden)

        self.out = nn.Linear(n_hidden, 1)

    def forward(self, edge_index, r_mat_edge_index, abstract_embeddings):
        edge_index_norm = gcn_norm(edge_index=edge_index,
                                   add_self_loops=self.add_self_loops)

        emb_0 = torch.cat([self.users_emb.weight, self.items_emb.weight, self.meta_nodes_emb.weight]) 

        embs = [emb_0]

        emb_k = emb_0

        for i in range(self.K):
            emb_k = self.propagate(edge_index=edge_index_norm[0], x=emb_k, norm=edge_index_norm[1])
            embs.append(emb_k)


        embs = torch.stack(embs, dim=1)

        emb_final = torch.mean(embs, dim=1) 

        users_emb_final, items_emb_final, _ = torch.split(emb_final,
                                                       [self.num_users, self.num_items, self.num_meta_nodes]) 

        src, dest =  r_mat_edge_index[0], r_mat_edge_index[1]
        # Here we only extract the users and items final embeddings 
        # The information of the meta nodes embeddings still has passed through them though propagate
        user_embeds = users_emb_final[src]
        item_embeds = items_emb_final[dest]

        x = torch.cat([user_embeds, item_embeds, abstract_embeddings], dim=1)

        x = self.drop1(x)

        x = F.relu(self.lin1(x))

        x = self.drop2(x)
        
        output = self.out(x)

        return output

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j


In [25]:
def get_abstract_embeddings_tensors(movies: torch.Tensor, idx2movieId):
    result = [[0] * 384]

    for i in range(1, len(idx2movieId)):
        movieId = idx2movieId.get(i, 0)
        result.append(movie2abstract_embed[movieId])
    return torch.tensor(result)

In [26]:
def get_meta_nodes_embeddings_tensors(movies: torch.Tensor, idx2movieId):
    result = [[0] * num_meta_nodes]

    for i in range(1, len(idx2movieId)):
        movieId = idx2movieId.get(i, 0)
        if movieId != 0:
            result.append(movie2embedding[movieId])
        else:
            result.append(np.zeros(num_meta_nodes, dtype=np.float32))
    return torch.tensor(result)

In [27]:
def get_fold_data(X_train, X_test, X_meta, get_abs_tensors):
    users_train = X_train.userId.unique()
    movies_train = X_train.movieId.unique()
    meta_nodes_unique = X_meta.metaNodeURI.unique()
    userid2idx = {o:i + 1 for i,o in enumerate(users_train)}
    movieid2idx = {o:i + 1 for i,o in enumerate(movies_train)}
    meta_node_uri2idx = {o:i + 1 for i,o in enumerate(meta_nodes_unique)}
    userid2idx[-1] = 0 # unk user
    movieid2idx[-1] = 0 # unk movie
    meta_node_uri2idx[-1] = 0 # unk meta
    idx2movieId= {v:k  for k,v in movieid2idx.items()}
    num_users, num_movies, num_meta_nodes = len(userid2idx), len(movieid2idx), len(meta_node_uri2idx)
    edge_index_train, edge_values_train = get_edge_index(X_train, X_meta, userid2idx, movieid2idx, meta_node_uri2idx)
    edge_index_test, edge_values_test = get_edge_index(X_test, X_meta, userid2idx, movieid2idx, meta_node_uri2idx)
    edge_index_train, edge_values_train = torch.tensor(edge_index_train), torch.tensor(edge_values_train)
    edge_index_test, edge_values_test   = torch.tensor(edge_index_test), torch.tensor(edge_values_test)
    adj_edge_index_train, adj_edge_values_train  = convert_r_mat_edge_index_to_adj_mat_edge_index(edge_index_train, edge_values_train, num_users, num_meta_nodes, num_movies)
    adj_edge_index_test, adj_edge_values_test   = convert_r_mat_edge_index_to_adj_mat_edge_index(edge_index_test, edge_values_test, num_users, num_meta_nodes, num_movies)
    r_mat_edge_index_train, r_mat_edge_values_train   = convert_adj_mat_edge_index_to_r_mat_edge_index(adj_edge_index_train, adj_edge_values_train, num_users, num_meta_nodes, num_movies)
    r_mat_edge_index_test,  r_mat_edge_values_test    = convert_adj_mat_edge_index_to_r_mat_edge_index(adj_edge_index_test, adj_edge_values_test, num_users, num_meta_nodes, num_movies)
    meta_nodes_embeddings_train = get_meta_nodes_embeddings_tensors(r_mat_edge_index_train[0, :], idx2movieId)
    meta_nodes_embeddings_test = get_meta_nodes_embeddings_tensors(r_mat_edge_index_test[0, :], idx2movieId)
    if get_abs_tensors:
        abstract_embeddings_train = get_abstract_embeddings_tensors(r_mat_edge_index_train[0, :], idx2movieId)
        abstract_embeddings_test = get_abstract_embeddings_tensors(r_mat_edge_index_test[0, :],idx2movieId)
    else:
        abstract_embeddings_train = get_abstract_embeddings(r_mat_edge_index_train[0, :], idx2movieId)
        abstract_embeddings_test = get_abstract_embeddings(r_mat_edge_index_test[0, :], idx2movieId)

    return  adj_edge_index_train,\
        r_mat_edge_index_train,\
        r_mat_edge_values_train,\
        abstract_embeddings_train,\
        adj_edge_index_test,\
        r_mat_edge_index_test,\
        r_mat_edge_values_test,\
        abstract_embeddings_test,\
        meta_nodes_embeddings_train,\
        meta_nodes_embeddings_test,\
        num_users,\
        num_movies,\
        num_meta_nodes


In [28]:
def get_deep_LightGCN_model(**kwargs):
    return DeepLightGCN(**kwargs)

In [31]:
def cv_model(get_model, get_abs_tensors=False, **kwargs):

    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    train_losses = []
    test_losses = []

    for fold, (X_train, X_test) in enumerate(kfold.split(ratings[['userId', 'movieId', 'rating']])):
        X_train = ratings[['userId', 'movieId', 'rating']].loc[X_train]
        X_test = ratings[['userId', 'movieId', 'rating']].loc[X_test]
        X_meta = all_metadata_df
        adj_edge_index_train,\
            r_mat_edge_index_train,\
            r_mat_edge_values_train,\
            abstract_embeddings_train,\
            adj_edge_index_test,\
            r_mat_edge_index_test,\
            r_mat_edge_values_test,\
            abstract_embeddings_test,\
            meta_nodes_embeddings_train,\
            meta_nodes_embeddings_test,\
            num_users,\
            num_movies,\
            num_meta_nodes = get_fold_data(X_train, X_test, X_meta, get_abs_tensors)
        model = get_model(num_users=num_users, num_items=num_movies, num_meta_nodes=num_meta_nodes, **kwargs)
        model.__init__(num_users=num_users, num_items=num_movies, num_meta_nodes=num_meta_nodes, **kwargs)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.01)
        scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
        start_time = timer()
        results = train(model=model,
            optimizer = optimizer,
            loss_fn = nn.MSELoss(),
            epochs = 10_000,
            edge_index_train=adj_edge_index_train,
            r_mat_edge_index_train=r_mat_edge_index_train,
            r_mat_edge_values_train=r_mat_edge_values_train,
            abstract_embeddings_train=abstract_embeddings_train,
            meta_nodes_embeddings_train=meta_nodes_embeddings_train,
            edge_index_test=adj_edge_index_test,
            r_mat_edge_index_test=r_mat_edge_index_test,
            r_mat_edge_values_test=r_mat_edge_values_test,
            abstract_embeddings_test=abstract_embeddings_test,
            meta_nodes_embeddings_test=meta_nodes_embeddings_test,
            device = device,
            scheduler=scheduler)
        print(len(results['train_loss']))
        end_time = timer()

        train_losses.append(min(results['train_loss']))
        test_losses.append(min(results['test_loss']))
        
        del model
        gc.collect()
        torch.cuda.empty_cache() 

        print(
            f"Fold: {fold} | "
            f"train_loss: {train_losses[-1]:.4f} | "
            f"train RMSE: {np.sqrt(train_losses[-1]):.4f} | "
            f"test_loss: {test_losses[-1]:.4f} | "
            f"test RMSE: {np.sqrt(test_losses[-1]):.4f} | "
            f"time: {(end_time-start_time):.4f} "
            )

    print(
        f"train_loss: {np.average(train_losses):.4f} | "
        f"train RMSE: {np.sqrt(np.average(train_losses)):.4f} | "
        f"test_loss: {np.average(test_losses):.4f} | "
        f"test RMSE: {np.sqrt(np.average(test_losses)):.4f} | "
                )                                                                                                                           

In [33]:
cv_model(get_deep_LightGCN_model, embedding_dim=128, K=1, n_hidden=256, add_self_loops=True, dropout_rate=0.5, name="deep_light_gcn_with_abs_meta_emb_v3_cv_v1")

  0%|          | 0/10000 [00:00<?, ?it/s]

10000
Fold: 0 | train_loss: 0.7077 | train RMSE: 0.8413 | test_loss: 0.7681 | test RMSE: 0.8764 | time: 546.0070 


  0%|          | 0/10000 [00:00<?, ?it/s]

10000
Fold: 1 | train_loss: 0.7113 | train RMSE: 0.8434 | test_loss: 0.7424 | test RMSE: 0.8616 | time: 696.2218 


  0%|          | 0/10000 [00:00<?, ?it/s]

10000
Fold: 2 | train_loss: 0.7105 | train RMSE: 0.8429 | test_loss: 0.7319 | test RMSE: 0.8555 | time: 519.4173 


  0%|          | 0/10000 [00:00<?, ?it/s]

10000
Fold: 3 | train_loss: 0.7137 | train RMSE: 0.8448 | test_loss: 0.7446 | test RMSE: 0.8629 | time: 541.9113 


  0%|          | 0/10000 [00:00<?, ?it/s]

10000
Fold: 4 | train_loss: 0.7072 | train RMSE: 0.8410 | test_loss: 0.7612 | test RMSE: 0.8725 | time: 524.7410 
train_loss: 0.7101 | train RMSE: 0.8427 | test_loss: 0.7496 | test RMSE: 0.8658 | 


In [34]:
class DeepLightGCN_2(MessagePassing):

    def save(self, fileName):
        torch.save(self.state_dict(), fileName)

    def load(self, fileName, device):
        self.load_state_dict(torch.load(fileName, map_location=device))

    def __init__(self, num_users, num_items, num_meta_nodes, embedding_dim=64, K=3, abstract_embedding_dim=384, n_hidden=32, add_self_loops=False, dropout_rate=0.1, name='name'):

        super().__init__()
        self.dropout_rate = dropout_rate
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.K = K
        self.add_self_loops = add_self_loops
        self.name = name
        self.drop1 = nn.Dropout(dropout_rate)
        self.drop2 = nn.Dropout(dropout_rate)
        self.abstract_embedding_dim = abstract_embedding_dim
        self.num_meta_nodes = num_meta_nodes

        self.users_emb = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.embedding_dim)

        self.items_emb = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.embedding_dim)
        self.meta_nodes_emb = nn.Embedding(num_embeddings=self.num_meta_nodes, embedding_dim=self.embedding_dim)

        nn.init.normal_(self.users_emb.weight, std=0.1)
        nn.init.normal_(self.items_emb.weight, std=0.1)
        nn.init.normal_(self.meta_nodes_emb.weight, std=0.1)
        self.lin1 = nn.Linear(embedding_dim*2, n_hidden)
        self.lin_abs2embedding = nn.Linear(abstract_embedding_dim, embedding_dim)

        self.out = nn.Linear(n_hidden, 1)

    def forward(self, edge_index, r_mat_edge_index, abstract_embeddings):
        edge_index_norm = gcn_norm(edge_index=edge_index,
                                   add_self_loops=self.add_self_loops)

        items_abstract_embeddings = self.lin_abs2embedding(abstract_embeddings)
        emb_0 = torch.cat([self.users_emb.weight, self.items_emb.weight, self.meta_nodes_emb.weight, items_abstract_embeddings])

        embs = [emb_0]

        emb_k = emb_0

        for i in range(self.K):
            emb_k = self.propagate(edge_index=edge_index_norm[0], x=emb_k, norm=edge_index_norm[1])
            embs.append(emb_k)

        embs = torch.stack(embs, dim=1)

        emb_final = torch.mean(embs, dim=1) 
        users_emb_final, items_emb_final, _, _ = torch.split(emb_final,
                                                       [self.num_users, self.num_items, self.num_meta_nodes, self.num_items]) 

        src, dest =  r_mat_edge_index[0], r_mat_edge_index[1]

        user_embeds = users_emb_final[src]
        item_embeds = items_emb_final[dest]

        x = torch.cat([user_embeds, item_embeds], dim=1)

        x = self.drop1(x)

        x = F.relu(self.lin1(x))

        x = self.drop2(x)
        
        output = self.out(x)

        return output

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j


In [35]:
def get_deep_LightGCN_v2_model(**kwargs):
    return DeepLightGCN_2(**kwargs)

In [37]:
cv_model(get_deep_LightGCN_v2_model, get_abs_tensors=True, embedding_dim=128, K=1, n_hidden=256, add_self_loops=True, dropout_rate=0.5, name="deep_light_gcn_with_abs_meta_emb_v3_cv_v2")

  0%|          | 0/10000 [00:00<?, ?it/s]

10000
Fold: 0 | train_loss: 0.7095 | train RMSE: 0.8423 | test_loss: 0.7682 | test RMSE: 0.8765 | time: 352.4351 


  0%|          | 0/10000 [00:00<?, ?it/s]

10000
Fold: 1 | train_loss: 0.7142 | train RMSE: 0.8451 | test_loss: 0.7420 | test RMSE: 0.8614 | time: 351.4090 


  0%|          | 0/10000 [00:00<?, ?it/s]

10000
Fold: 2 | train_loss: 0.7133 | train RMSE: 0.8446 | test_loss: 0.7321 | test RMSE: 0.8556 | time: 352.4288 


  0%|          | 0/10000 [00:00<?, ?it/s]

10000
Fold: 3 | train_loss: 0.7109 | train RMSE: 0.8432 | test_loss: 0.7440 | test RMSE: 0.8626 | time: 358.3880 


  0%|          | 0/10000 [00:00<?, ?it/s]

10000
Fold: 4 | train_loss: 0.7051 | train RMSE: 0.8397 | test_loss: 0.7609 | test RMSE: 0.8723 | time: 350.2259 
train_loss: 0.7106 | train RMSE: 0.8430 | test_loss: 0.7494 | test RMSE: 0.8657 | 
